## 2. The IntervalArithmetic.jl package

Now that we have covered some basic ideas of interval arithmetic, you should be able, in principle, to implement each operation. But guaranteeing that the results are rounded correctly, getting all the corner cases right, with generic code, all while achieving high performance, is non-trivial.

Together with Luis Benet (ICF-UNAM, Mexico) we have been developing the `IntervalArithmetic.jl` package since 2014. Within the next couple of months we expect it to be compliant with the IEEE-1788 Standard document, which specifies how interval arithmetic packages should behave. It passes a large, independently-developed test suite, ITF1788. 

In order to get to some of the interesting applications of these methods, we will now switch to using the `IntervalArithmetic.jl` package that we have been developing 

As usual, we install the package with

In [ ]:
] add IntervalArithmetic

and load it with

In [3]:
using IntervalArithmetic

This is a complete implementation of interval arithmetic that is very fast for `Float64` intervals, but also allows other floating-point types, in particular `BigFloat` (which are slow, because `BigFloat`s are slow).

The two main ways to create an interval are as follows:

In [2]:
X1 = interval(0.1, 0.3)  # note: lower-case `i`

[0.1, 0.3]

In [3]:
X2 = 0.1..0.3    # .. operator

[0.0999999, 0.300001]

Note that these give slightly different results: the `..` operator currently tries to be "clever" and guess "what you really meant": in this case you supposedly "really meant" the true real number 0.1, not the floating-point number `0.1`. However, we are strongly considering removing this perhaps-too-clever behaviour.

All the standard arithmetic and elementary functions have methods defined for intervals. For example:

In [5]:
X1 + X2

[0.199999, 0.600001]

This means that you can use standard Julia functions with intervals as follows:

In [6]:
f(x) = x^2 + 2x

f (generic function with 1 method)

In [7]:
X = -1..1

[-1, 1]

In [8]:
f(X)

[-2, 3]

By the fundamental theorem of interval arithmetic, this result is guaranteed to be an enclosure of the true range of the function over the set. However, in this case, even for such an apparently simple function, it turns out to be an *over*-approximation.

#### Exercise

1. Define functions `f1` and `f2` giving equivalent algebraic versions of the function (by factorising etc.). Do they give better or worse ranges? Can you find the exact range?

## Set operations

Since intervals are sets, it also makes sense to define standard set operations. Since this is Julia, we can *use the correct mathematical operators* for this!

In [6]:
(1..3) ∩ (2..4)

[2, 3]

This makes the code so nice to read and understandable.

#### Exercise

How can you calculate the set difference of two intervals? What should it give you? What does it give you?

## Kahan's example

Let's go back to Kahan's example from the start of the last notebook.

In [9]:
f(x) = (1/80) * log(abs(3*(1 - x) + 1)) + x^2 + 1

f (generic function with 1 method)

Since all of these functions are defined for intervals, we can find an enclosure of the range of this function over a given input interval $X$ by just evaluating the function using that input interval:

In [10]:
f(1.0..1.2)

[1.98854, 2.44001]

Now let's try the key region:

In [11]:
f(1.3..1.4)

[-∞, 2.93989]

We see that interval arithmetic manages what we were previously unable to do: It warns us that something weird *might* be happening in this interval. But note that since this is only guaranteed to be an *over*-approximation, it does not guarantee that the range actually goes all the way to $-\infty$; that is something you can work out only by analytical means.

## Improving enclosures: Bounding functions

How can we improve an *over*-estimate of a range? It turns out that we can decrease the amount of over-estimation by simply splitting up the initial interval into pieces. This is sometimes called **mincing** the interval.

#### Exercise

1. Write a function `mymince` to split a given interval $X$ into $N$ equal-sized pieces.


2. For the function $f(x) = x^2 + 2x$ visualize the input and output intervals.


3. Make this into an interactive visualization with `Interact.jl` where you vary the number $N$ of sub-intervals.s


4. Apply this to the Kahan function. You will need to check for infinity and cut off the result at a finite value.

## Application: Numerical integration

A nice, simple application of interval arithmetic is to calculate integrals of functions numerically. For simplicity let's restrict ourselves to functions $f: \mathbb{R} \to \mathbb{R}$, although in principal the same idea extends to higher dimensions.

Recall that the integral
$$I := \int_a^b f(x) \, dx$$
represents the set $A$ between the function $f$ and the $x$-axis between $a$ and $b$.
    
We can approximate this by splitting up the interval $[a, b]$ into pieces and using a set of rectangles of the right height. Which is the right height? Standard numerical methods take e.g. $f(m)$, where $m$ is the midpoint.

Instead, having intervals at our disposal, we can *evaluate $f$ over the whole interval*! This will give us guaranteed lower and upper bounds for $f$ over that interval. Taking those lower and upper bounds gives two possible rectangles: one which is contained in $A$, and one which contains the part of $A$ that lies between $a$ and $b$.

In other words, interval arithmetic immediately gives us Riemann lower and upper sums, and hence rigorous bounds for $I$!

#### Exercise

1. Implement this. Apply it to different functions.


2. How fast does the approximation improve as you take more sub-intervals?

## Formatting

The `@format` macro changes the way that intervals are displayed:

In [15]:
X = 0.1..0.3  # default is standard mode with 6 significant figures

Interval(0.09999999999999999, 0.30000000000000004)

In [11]:
@format midpoint 3

Display parameters:
- format: midpoint
- decorations: false
- significant figures: 3

In [12]:
X

0.2 ± 0.101

In [16]:
@format full

Display parameters:
- format: full
- decorations: false
- significant figures: 10

In [17]:
X

Interval(0.09999999999999999, 0.30000000000000004)

In [18]:
@format standard 10

Display parameters:
- format: standard
- decorations: false
- significant figures: 10

In [19]:
X

[0.09999999999, 0.3000000001]

In standard mode the displayed result is always an *enclosure* of the interval.
In `full` mode the floating-point endpoints are displayed using the usual Julia algorithm.